In [23]:
import csv
import time
from collections import defaultdict
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

# === CẤU HÌNH SELENIUM ===
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--disable-blink-features=AutomationControlled')
driver = webdriver.Chrome(options=options)

def wait_and_click(by, value, timeout=30):
    try:
        WebDriverWait(driver, timeout).until(EC.element_to_be_clickable((by, value))).click()
        time.sleep(0.5)
    except Exception as e:
        print(f"❌ Không thể click: {value}")
        raise e

def wait_and_send_keys(by, value, text, timeout=30):
    WebDriverWait(driver, timeout).until(EC.element_to_be_clickable((by, value))).send_keys(text)
    time.sleep(0.2)

def extract_domain(email):
    return email.split('@')[1].lower().strip() if '@' in email else None

def find_or_create_company(domain, company_hint=""):
    wait_and_click(By.XPATH, "//i[@class='oi oi-apps']")
    wait_and_click(By.XPATH, "//a[contains(text(), 'Email Marketing')]")
    time.sleep(1)
    wait_and_click(By.XPATH, "//a[contains(text(), 'Companies')]")
    time.sleep(1)

    keyword = company_hint.strip() or domain
    search_input = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "//input[@placeholder='Search...']"))
    )
    search_input.clear()
    search_input.send_keys(keyword)
    search_input.send_keys(Keys.ENTER)
    print(f"🔍 Tìm công ty: {keyword}")
    time.sleep(2.5)

    try:
        rows = driver.find_elements(By.XPATH, "//table//tbody/tr")
        if rows:
            rows[0].click()
            print(f"✅ Đã chọn công ty: {rows[0].text.strip()}")
            time.sleep(1.5)
            return
    except Exception as e:
        print(f"⚠️ Lỗi khi chọn công ty: {e}")

    print("➕ Không thấy công ty phù hợp, tạo mới...")
    wait_and_click(By.XPATH, "//button[contains(@class, 'o_list_button_add')]")
    wait_and_send_keys(By.NAME, "name", company_hint or domain)
    WebDriverWait(driver, 20).until(EC.invisibility_of_element_located((By.CLASS_NAME, "o_blockUI")))
    wait_and_click(By.XPATH, "//button[.//span[contains(text(), 'Save')] or contains(text(), 'Save')]")
    print(f"✅ Tạo mới công ty: {company_hint or domain}")

def is_email_existing_in_company(email):
    xpath = f"//table//tr//td[contains(text(), '{email.lower()}')]"
    try:
        WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, xpath)))
        return True
    except:
        return False

def add_contact_inline_in_company(name, email, has_next):
    print(f"👤 Thêm contact: {name} - {email}")
    wait_and_click(By.XPATH, "//a[contains(text(),'Add a line') or contains(@class, 'o_form_button_add')]")

    WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, "//h4[contains(text(), 'Create Contact')]"))
    )

    popup = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//div[contains(@class,'modal-content')]"))
    )

    # Điền Tên
    name_input = WebDriverWait(popup, 10).until(
        EC.visibility_of_element_located((By.XPATH, ".//input[contains(@placeholder, 'Brandon Freeman')]"))
    )
    name_input.clear()
    name_input.send_keys(name)

    # Điền Email (có fallback nếu không tìm thấy bằng name='email')
    try:
        email_input = WebDriverWait(popup, 5).until(
            EC.visibility_of_element_located((By.XPATH, ".//input[@name='email']"))
        )
    except:
        print("⚠️ Không tìm thấy input[name='email'], thử theo label...")
        email_input = WebDriverWait(popup, 5).until(
            EC.visibility_of_element_located((
                By.XPATH,
                ".//label[contains(text(), 'Email')]/following::input[1]"
            ))
        )

    email_input.clear()
    email_input.send_keys(email)
    time.sleep(0.3)  # Cho Odoo xử lý input email

    # Lưu contact
    if has_next:
        btn = WebDriverWait(popup, 10).until(
            EC.element_to_be_clickable((By.XPATH, ".//button[normalize-space(.)='Save & New']"))
        )
        print("🔁 Còn contact khác, chọn Save & New\n")
    else:
        btn = WebDriverWait(popup, 10).until(
            EC.element_to_be_clickable((By.XPATH, ".//button[normalize-space(.)='Save & Close']"))
        )
        print("✅ Contact cuối cùng, chọn Save & Close\n")

    btn.click()

    # ✅ Chờ popup đóng nếu là Save & Close
    if not has_next:
        WebDriverWait(driver, 10).until(
            EC.invisibility_of_element_located((By.XPATH, "//div[contains(@class,'modal-content')]"))
        )
        print("✅ Đã đóng popup sau khi Save & Close")

    time.sleep(0.5)
    print(f"✅ Đã lưu contact: {name} - {email}\n")

def import_contacts_grouped_by_domain(csv_path):
    grouped = defaultdict(list)

    with open(csv_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            name = row.get("name") or row.get("Name") or ""
            email = row.get("email") or row.get("Email") or ""
            company_hint = row.get("company") or row.get("Company") or row.get("Related Company") or ""

            if not email or '@' not in email:
                print(f"⚠️ Bỏ qua contact không hợp lệ: {name}")
                continue

            domain = extract_domain(email)
            if not domain:
                print(f"⚠️ Không thể lấy domain từ email: {email}")
                continue

            grouped[(domain, company_hint)].append((name, email))

    for (domain, company_hint), contact_list in grouped.items():
        find_or_create_company(domain, company_hint)
        total = len(contact_list)
        for i, (name, email) in enumerate(contact_list):
            if is_email_existing_in_company(email):
                print(f"⚠️ Email `{email}` đã tồn tại, bỏ qua.")
                continue
            has_next = i < total - 1
            add_contact_inline_in_company(name, email, has_next)

        # ✅ Sau khi thêm tất cả contact → bấm nút Save
        wait_and_click(By.XPATH, "//button[contains(@class, 'o_form_button_save')]")
        print("💾 Đã ấn nút Save ở form công ty\n")


# === THỰC THI ===
driver.get('https://crm2.wsoftpro.com/web/login')
wait_and_send_keys(By.ID, "login", "trung@wsoftpro.com")
wait_and_send_keys(By.ID, "password", "wsoftpro123")
wait_and_click(By.XPATH, "//button[@type='submit' and contains(text(), 'Log in')]")

time.sleep(4)
import_contacts_grouped_by_domain("C:/Users/hoang/OneDrive/Documents/contact_n.csv")
driver.quit()


🔍 Tìm công ty: FOXMINDS Solutions
✅ Đã chọn công ty: FOXMINDS Solutions 03/31/2025 10:02:34
⚠️ Email `lpahdph432e1572@foxminds.com` đã tồn tại, bỏ qua.
⚠️ Email `hoangd32uyl334ap1124@foxminds.com` đã tồn tại, bỏ qua.
⚠️ Email `4133@foxminds.com` đã tồn tại, bỏ qua.
❌ Không thể click: //button[contains(@class, 'o_form_button_save')]


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=136.0.7103.114)
Stacktrace:
	GetHandleVerifier [0x00007FF750D2CF45+75717]
	GetHandleVerifier [0x00007FF750D2CFA0+75808]
	(No symbol) [0x00007FF750AF8F9A]
	(No symbol) [0x00007FF750AE4E35]
	(No symbol) [0x00007FF750B09DB4]
	(No symbol) [0x00007FF750B7EE75]
	(No symbol) [0x00007FF750B9ECC2]
	(No symbol) [0x00007FF750B77153]
	(No symbol) [0x00007FF750B40421]
	(No symbol) [0x00007FF750B411B3]
	GetHandleVerifier [0x00007FF75102D71D+3223453]
	GetHandleVerifier [0x00007FF751027CC2+3200322]
	GetHandleVerifier [0x00007FF751045AF3+3322739]
	GetHandleVerifier [0x00007FF750D46A1A+180890]
	GetHandleVerifier [0x00007FF750D4E11F+211359]
	GetHandleVerifier [0x00007FF750D35294+109332]
	GetHandleVerifier [0x00007FF750D35442+109762]
	GetHandleVerifier [0x00007FF750D1BA59+4825]
	BaseThreadInitThunk [0x00007FFB67E8E8D7+23]
	Rtl